## QIB Email *Extract, Load, and Transform* Notebook

### Imports

In [1]:
# Standard Library Imports
import logging
import time
from typing import List
from multiprocessing import Pool, Manager, cpu_count

# Third-Party Imports
import pypff # type: ignore
from sqlalchemy.exc import SQLAlchemyError
from tqdm.notebook import tqdm

# Local Imports
from src.config.config import Config
from src.extract.message_extractor import PstMessageExtractor, MessageBatch, PstMessage
from src.transform.message_parser import MessageParser, ParsedMessage
from src.transform.message_enricher import MessageEnricher, EnrichedMessage
from src.load.data_loader import DataLoader
from src.database.database import Database
from src.database.export_utils import DataExporter

logging.basicConfig(level=logging.INFO)
logging.getLogger('chardet.charsetprober').disabled = True

### Configuration

In [2]:
config = Config.from_json("config.json")
extractor = PstMessageExtractor(config.input_pst_path, config.chunk_size)
message_parser = MessageParser()
message_enricher = MessageEnricher()
database = Database.from_credentials(username=config.db_user, password=config.db_password, host=config.db_host, database=config.db_name)
loader = DataLoader(database)
exporter = DataExporter(database)

### Define Worker Function for Multiprocessing

In [3]:
def process_message_batch(message_batch: MessageBatch):
    parsed_messages = []
    enriched_messages = []
    loaded_count = 0

    for pst_message in message_batch.messages:
        try:
            parsed_message = message_parser.parse(pst_message.message, pst_message.folder_name)
            parsed_messages.append(parsed_message)
        except Exception as e:
            logging.error(f"Error parsing message: {e}")
            continue

    for parsed_message in parsed_messages:
        try:
            enriched_message = message_enricher.enrich_message(parsed_message)
            enriched_messages.append(enriched_message)
        except Exception as e:
            logging.error(f"Error enriching message: {e}")
            continue

    for enriched_message in enriched_messages:
        try:
            loader.load(enriched_message)
            loaded_count += 1
        except SQLAlchemyError as e:
            logging.error(f"Error loading message {enriched_message.provider_email_id}: {e}")
            continue

    return loaded_count

### ETL Pipeline with Multiprocessing

In [4]:
total_messages = extractor.get_total_messages("Inbox")
start_time = time.time()

database.drop_all_tables()
loader.create_tables()
logging.info("Starting ETL Pipeline")

with Pool(cpu_count()) as pool:
    results = pool.imap_unordered(process_message_batch, extractor.extract_messages("Inbox"))
    
    with tqdm(total=total_messages, desc='Processing messages') as pbar:
        for loaded_count in results:
            pbar.update(loaded_count)

end_time = time.time()
logging.info(f"Finished ETL Pipeline in {int(end_time - start_time) // 60} minutes and {int(end_time - start_time) % 60} seconds")

INFO:root:All tables have been dropped from the database.
INFO:root:All tables have been created in the database.
INFO:root:Starting ETL Pipeline


Processing messages:   0%|          | 0/5039 [00:00<?, ?it/s]

TypeError: cannot pickle 'pypff.message' object

In [ ]:
exporter.export_to_excel("./data/processed")

In [ ]:
exporter.export_schema('./data')